1. Remember user preferences and past conversations
2. Store and retrieve factual information about users and topics
3. Use memory to inform decisions and provide personalized responses
4. Handle complex multi-turn conversations with context awareness
5. Manage different types of memories (facts, preferences, experiences)

In [ ]:
!pip install -q dspy mem0ai

### 1. MemO integration for storing, searching and retrieving memories for agents

In [ ]:
import dspy 
from mem0 import Memory 
import os 
from typing import List, Dict, Any, Optional 
from datetime import datetime 

#config the environment 

os.environ["OPENAI_API_KEY"]= "api_key"

#initilaise mem0 memory system 

config={
    "llm":{
        "provider": "openai", 
        "config":{
            "model":"gpt-4o-mini", 
            "temperature": 0.1
        }
    }, 
    "embedder":{
        "provider": "openai", 
        "config":{
            "model":"text-embedding-3-small"
        }
    }
}


### 2. memory aware tools

In [ ]:
import datetime

class MemoryTools:
    """
    Tools for interacting with mem0 memory system.
    """

    def __init__(self, memory: Memory):  #the : is used for type hints by the typing module in python 
        self.memory=memory

    def store_memory(self, content: str, user_id: str ="default_user") -> str: 
        """
        store information in the memory
        """

        try: 
            self.memory.add(content, user_id= user_id)
            return f" stored memory for user {user_id} as {content}"

        except Exception as e: 
            return f" error storing memory {str(e)}"
        
    
    def search_memories(self, query: str, user_id: str="default_user",limit: int = 5) -> str:
        """
        search for relvent memories from the db 
        """

        try: 
            results=self.memory.search(query, user_id= user_id, limit=limit)
            if not results: 
                return "No memories found for the user"
            
            memory_text="All memories of the user:\n"
            for idx, result in enumerate( results['results']):
                memory_text+= f"{idx+1}. {result['content']} (Added on: {result['created_at']})\n"
            return memory_text
        
        except Exception as e: 
            return f" error searching memories {str(e)}"
    
    def update_memory(self, memory_id: str, new_content: str) -> str:
        """Update an existing memory."""
        try:
            self.memory.update(memory_id, new_content)
            return f"Updated memory with new content: {new_content}"
        except Exception as e:
            return f"Error updating memory: {str(e)}"

    def delete_memory(self, memory_id: str) -> str:
        """Delete a specific memory."""
        try:
            self.memory.delete(memory_id)
            return "Memory deleted successfully."
        except Exception as e:
            return f"Error deleting memory: {str(e)}"

def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")




### 3. Building memory enhanced ReAct Agent 

In [ ]:
class MemoryQA(dspy.Signature):
    """
    You're a helpful assistant and have access to memory method.
    Whenever you answer a user's input, remember to store the information in memory
    so that you can use it later.
    """
    user_input: str= dspy. InputField()
    response: str= dspy.OutputField()


class MemoryReActAgent(dspy.Module):
    """
    ReACT agent enhanced with Mem0 memory capabilities
    """

    def __init__(self, memory: Memory): 

        super().__init__()
        self.memory_tools= MemoryTools(memory)

        #create tools list for react 

        self.tools=[
            self.memory_tools.store_memory,
            self.memory_tools.search_memories,
            self.memory_tools.get_all_memories,
            get_current_time,
            self.set_reminder,
            self.get_preferences,
            self.update_preferences,
                    
             ]
        
        #initilase react wioth our tools 

        self.react=dspy.ReAct(
            signature=MemoryQA, 
            tools=self.tools, 
            max_iters=6
        )

    def forward(self, user_input: str):
            "process user input with memory aware reasoning"
            return self.react(user_input=user_input) 
        
    def set_reminder(self, reminder_text: str, date_time: str = None, user_id: str = "default_user") -> str:
        """Set a reminder for the user."""
        reminder = f"Reminder set for {date_time}: {reminder_text}"
        return self.memory_tools.store_memory(
            f"REMINDER: {reminder}", 
            user_id=user_id
        )

    def get_preferences(self, category: str = "general", user_id: str = "default_user") -> str:
        """Get user preferences for a specific category."""
        query = f"user preferences {category}"
        return self.memory_tools.search_memories(
            query=query,
            user_id=user_id
        )

    def update_preferences(self, category: str, preference: str, user_id: str = "default_user") -> str:
        """Update user preferences."""
        preference_text = f"User preference for {category}: {preference}"
        return self.memory_tools.store_memory(
            preference_text,
            user_id=user_id
        )


### 4. running react agent 

In [ ]:
import time 
def run_memory_agent():
    """
    run demo of mem0 react agent dspy 
    """

    lm=dspy.LM(model='openai/gpt-4o-mini')
    dspy.configure(lm=lm)

    memory=Memory.from_config(config)

    agent=MemoryReActAgent(memory)

     # Sample conversation demonstrating memory capabilities
    print(" Memory-Enhanced ReAct Agent Demo")
    print("=" * 50)

    conversations = [
        "Hi, I'm Alice and I love Italian food, especially pasta carbonara.",
        "I'm Alice. I prefer to exercise in the morning around 7 AM.",
        "I'm Alice. What do you remember about my food preferences?",
        "I'm Alice. Set a reminder for me to go grocery shopping tomorrow.",
        "I'm Alice. What are my exercise preferences?",
        "I'm Alice. I also enjoy hiking on weekends.",
        "I'm Alice. What do you know about me so far?"
    ]

    for i, user_input in enumerate(conversations, 1):
        print(f"\n📝 User: {user_input}")

        try:
            response = agent(user_input=user_input)
            print(f"🤖 Agent: {response.response}")
            time.sleep(1)

        except Exception as e:
            print(f" Error: {e}")


if __name__=="__main__":

    run_memory_agent()


